<a href="https://colab.research.google.com/github/chaindmhl/BoardMate/blob/main/checker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install fastapi uvicorn python-multipart pyngrok opencv-python-headless numpy pyngrok

In [4]:
# Download model1
!wget -O model1.zip "https://github.com/chaindmhl/BoardMate/releases/download/v1.0/model1.zip"
!unzip -o model1.zip -d model1

# Download model2
!wget -O model2.zip "https://github.com/chaindmhl/BoardMate/releases/download/v1.0/model2.zip"
!unzip -o model2.zip -d model2


--2025-12-04 07:49:55--  https://github.com/chaindmhl/BoardMate/releases/download/v1.0/model1.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/1105783232/33f94dc6-316a-4681-a893-dca89576f10f?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-04T08%3A43%3A56Z&rscd=attachment%3B+filename%3Dmodel1.zip&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-04T07%3A43%3A17Z&ske=2025-12-04T08%3A43%3A56Z&sks=b&skv=2018-11-09&sig=7HmtDR2h%2BVtendte3u6NP%2BxglOh8koe%2F%2Bvz85TmaGhE%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NDgzODE5NSwibmJmIjoxNzY0ODM0NTk1LCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdGlvbi5ibG9iL

In [1]:
!fuser -k 5000/tcp


In [2]:
import base64
import io
import os
import json
import traceback
import threading
import cv2
import numpy as np
from flask import Flask, request, jsonify
from pyngrok import ngrok

app = Flask(__name__)

# ---------------- YOLO model loading ----------------
def load_yolo_model(model_dir, cfg_name, weights_name, names_name):
    cfg_file = os.path.join(model_dir, cfg_name)
    weights_file = os.path.join(model_dir, weights_name)
    names_file = os.path.join(model_dir, names_name)

    if not all(os.path.exists(f) for f in [cfg_file, weights_file, names_file]):
        raise FileNotFoundError("Missing YOLO files in " + model_dir)

    net = cv2.dnn.readNet(weights_file, cfg_file)
    with open(names_file) as f:
        classes = f.read().strip().split("\n")
    return net, classes

# Replace with your paths
NET_ORIGINAL, CLASSES_ORIGINAL = load_yolo_model("/content/model1/model1", "model1.cfg", "model1.weights", "model1.names")
NET_CROPPED, CLASSES_CROPPED = load_yolo_model("/content/model2/model2", "model2.cfg", "model2.weights", "model2.names")

# ---------------- Utility functions ----------------
def calculate_iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    xi1 = max(x1, x2)
    yi1 = max(y1, y2)
    xi2 = min(x1+w1, x2+w2)
    yi2 = min(y1+h1, y2+h2)
    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
    box1_area = w1*h1
    box2_area = w2*h2
    iou = inter_area / float(box1_area + box2_area - inter_area + 1e-6)
    return iou

def run_yolo_inference_with_nms(net, classes, img, conf_threshold=0.5, nms_threshold=0.4):
    h, w = img.shape[:2]
    blob = cv2.dnn.blobFromImage(img, 1/255.0, (416,416), swapRB=True, crop=False)
    net.setInput(blob)
    outputs = net.forward(net.getUnconnectedOutLayersNames())

    boxes, confidences, class_ids = [], [], []
    for output in outputs:
        for det in output:
            scores = det[5:]
            class_id = int(np.argmax(scores))
            confidence = float(scores[class_id])
            if confidence > conf_threshold:
                cx, cy, bw, bh = det[0:4] * np.array([w,h,w,h])
                x = int(cx - bw/2)
                y = int(cy - bh/2)
                boxes.append([x, y, int(bw), int(bh)])
                confidences.append(confidence)
                class_ids.append(class_id)

    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
    if len(indices) > 0:
        if isinstance(indices[0], (list, tuple, np.ndarray)):
            filtered_boxes = [boxes[i[0]] for i in indices]
            filtered_classes = [classes[class_ids[i[0]]] for i in indices]
            filtered_conf = [confidences[i[0]] for i in indices]
        else:  # 1D
            filtered_boxes = [boxes[i] for i in indices]
            filtered_classes = [classes[class_ids[i]] for i in indices]
            filtered_conf = [confidences[i] for i in indices]
    else:
        filtered_boxes, filtered_classes, filtered_conf = [], [], []

    return filtered_boxes, filtered_classes, filtered_conf

def calculate_distance(p1, p2):
    return np.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)

def draw_annotations(image, yolo1_pairs, yolo2_pairs, seq_dict):
    img = image.copy()
    for box, cls, conf in yolo1_pairs:
        x,y,w,h = box
        cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,0), 2)
        cv2.putText(img, cls, (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1)
    for box, cls, conf, seq in yolo2_pairs:
        x,y,w,h = box
        cv2.rectangle(img, (x,y), (x+w, y+h), (255,0,0), 2)
        cv2.putText(img, f"{seq}:{cls}", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 1)
    return img

# ---------------- Sort boxes by row-major (top-to-bottom, left-to-right) ----------------
def sort_boxes_row_major(boxes_with_cls):
    # boxes_with_cls: list of (box, cls, conf)
    # Sort by y first (top), then x (left)
    return sorted(boxes_with_cls, key=lambda x: (x[0][1], x[0][0]))

def sort_boxes_column_then_row(boxes):
    # boxes: [(box, cls, conf)]

    # 1. Sort all by x (left to right)
    boxes = sorted(boxes, key=lambda x: x[0][0])

    columns = []
    column_threshold = 50  # adjust if needed

    for item in boxes:
        x, y, w, h = item[0]
        placed = False

        for col in columns:
            # Check if x is close enough to belong to this column
            if abs(col[0][0][0] - x) < column_threshold:
                col.append(item)
                placed = True
                break

        if not placed:
            columns.append([item])

    # 2. Sort each column by y (top to bottom)
    for col in columns:
        col.sort(key=lambda x: x[0][1])

    # 3. Flatten back in column-major order
    sorted_final = []
    for col in columns:
        sorted_final.extend(col)

    return sorted_final


# ---------------- Flask endpoint ----------------
@app.route("/process_answer", methods=["POST"])
def process_answer():
    try:
        correct_answers = request.form.get("correct_answers")
        if correct_answers:
            try:
                correct_answers = json.loads(correct_answers)
            except:
                correct_answers = None

        file = request.files.get("image")
        npimg = np.frombuffer(file.read(), np.uint8)
        img_bgr = cv2.imdecode(npimg, cv2.IMREAD_COLOR)
        if img_bgr is None:
            return jsonify({"error":"failed to decode image"}), 400

        gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
        _, mask = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        mask3 = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
        img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

        # Detect answer boxes
        boxes1, names1, confs1 = run_yolo_inference_with_nms(NET_ORIGINAL, CLASSES_ORIGINAL, mask3)
        yolo1_pairs = [(box, name, conf) for box,name,conf in zip(boxes1, names1, confs1) if name=="answer"]

        # Detect letters in each answer box
        sorted_boxes_for_seq = []
        for box1, _, _ in yolo1_pairs:
            x,y,w,h = box1
            cropped = mask3[y:y+h, x:x+w]
            if cropped is None or cropped.size==0:
                continue
            boxes2, names2, confs2 = run_yolo_inference_with_nms(NET_CROPPED, CLASSES_CROPPED, cropped)
            for bx, cls, conf in zip(boxes2, names2, confs2):
                bx_full = [int(bx[0]+x), int(bx[1]+y), int(bx[2]), int(bx[3])]
                sorted_boxes_for_seq.append((bx_full, cls, conf))

        if not sorted_boxes_for_seq:
            return jsonify({"score":0, "total_items":0, "submitted_answers":{}, "annotated_image_base64":None})

        # Sort detected letters correctly
        sorted_boxes_for_seq = sort_boxes_column_then_row(sorted_boxes_for_seq)

        # Assign sequence numbers
        seq_num_class_dict = {}
        yolo2_pairs_for_draw = []
        for seq_num, (bx_full, cls, conf) in enumerate(sorted_boxes_for_seq, start=1):
            seq_num_class_dict[seq_num] = cls
            yolo2_pairs_for_draw.append((bx_full, cls, conf, seq_num))

        # Compute score
        score = 0
        if correct_answers:
            for seq, cls in seq_num_class_dict.items():
                if str(seq) in correct_answers and cls==correct_answers[str(seq)]:
                    score += 1

        annotated = draw_annotations(img_rgb, yolo1_pairs, yolo2_pairs_for_draw, seq_num_class_dict)
        annotated_bgr = cv2.cvtColor(annotated, cv2.COLOR_RGB2BGR)
        _, png = cv2.imencode('.png', annotated_bgr)
        b64 = base64.b64encode(png.tobytes()).decode('utf-8')

        submitted_answers_struct = {str(k): {"letter":v} for k,v in seq_num_class_dict.items()}

        return jsonify({
            "score": score,
            "total_items": len(seq_num_class_dict),
            "submitted_answers": submitted_answers_struct,
            "annotated_image_base64": b64
        })

    except Exception as e:
        traceback.print_exc()
        return jsonify({"error":str(e)}),500

# ---------------- Run Flask in background ----------------
def run_app():
    app.run(host="0.0.0.0", port=5000, debug=False, use_reloader=False)

thread = threading.Thread(target=run_app)
thread.daemon = True
thread.start()

ngrok_tunnel = ngrok.connect(5000)
public_url = ngrok_tunnel.public_url
print("Ngrok public URL:", public_url)
print("Full endpoint URL:", public_url + "/process_answer")


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


Ngrok public URL: https://kasi-releasible-conscionably.ngrok-free.dev
Full endpoint URL: https://kasi-releasible-conscionably.ngrok-free.dev/process_answer
